In [ ]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download("stopwords")


from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=pd.read_csv("/content/Grade 3 & 4 Dataset  - Grade 3 & 4 (1).csv", encoding="utf-8")
df.head(10)


,Sentences,class
0,මල පිපෙයි.,correct
1,ගඟ ගලයි.,correct
2,තරු බබළයි.,correct
3,කපුටෝ කෑගසති.,correct
4,මම කමි.,correct
5,නුඹ කන්නෙහි.,correct
6,ඔහු කයි.,correct
7,ඇය කයි.,correct
8,අපි කමු.,correct
9,ඔවුහු කති.,correct


In [ ]:
df.isnull().sum()
df.isna()
df.dropna(axis=0,inplace=True)
df.head()
print(df["class"].value_counts())

class
correct      1029
incorrect    1029
Name: count, dtype: int64


In [ ]:
def pre_text(tweet):
    ''' Convert tweet text into a sequence of words '''

    # convert to lowercase
    text = tweet.lower()
    # remove non letters
    #text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    #words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    #words = [PorterStemmer().stem(w) for w in words]
    # return list

    final_text=' '.join(words)
    return final_text

print("\nOriginal tweet ->", df['Sentences'][12])
print("\nProcessed tweet ->", pre_text(df['Sentences'][12]))


Original tweet -> ඔහු බොයි.

Processed tweet -> ඔහු බොයි.


In [ ]:
# Apply data processing to each tweet
pre_text_list = list(map(pre_text, df['Sentences']))

In [ ]:
from tensorflow.keras.utils import to_categorical
# softmax
# Map labels to integers
label_map = {"incorrect": 0, "correct": 1}  # assign 0/1
y_int= df["class"].map(label_map).astype(int).values
print("Integer labels:", y_int)

Integer labels: [1 1 1 ... 0 0 0]


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

model_name = "bert-base-multilingual-cased"   # works with Sinhala
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
train_enc = tokenizer(
    pre_text_list,
    padding="max_length",   # pad sequences
    truncation=True,        # truncate sequences longer than max_length
    max_length=8,           # limit to 4 tokens
    return_tensors="tf"
)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
labels = tf.constant(y_int)

In [ ]:

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    from_pt=True   # Force PyTorch → TensorFlow conversion
)


pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.fit(train_enc.data, labels, epochs=10, batch_size=8)

Epoch 1/10
258/258 [==============================] - 77s 87ms/step - loss: 0.7610 - accuracy: 0.4830
Epoch 2/10
258/258 [==============================] - 21s 80ms/step - loss: 0.7654 - accuracy: 0.4971
Epoch 3/10
258/258 [==============================] - 19s 72ms/step - loss: 0.7342 - accuracy: 0.4859
Epoch 4/10
258/258 [==============================] - 20s 76ms/step - loss: 0.7182 - accuracy: 0.4956
Epoch 5/10
258/258 [==============================] - 22s 87ms/step - loss: 0.7298 - accuracy: 0.5102
Epoch 6/10
258/258 [==============================] - 19s 72ms/step - loss: 0.7381 - accuracy: 0.4922
Epoch 7/10
258/258 [==============================] - 19s 75ms/step - loss: 0.7378 - accuracy: 0.5165
Epoch 8/10
258/258 [==============================] - 19s 73ms/step - loss: 0.7481 - accuracy: 0.4888
Epoch 9/10
258/258 [==============================] - 20s 76ms/step - loss: 0.7493 - accuracy: 0.5155
Epoch 10/10
258/258 [==============================] - 22s 86ms/step - loss: 0.751

## Prediction

In [ ]:
import numpy as np

def predict_sentence_tf(text):

    # Tokenize
    enc = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=8,
        return_tensors="tf"
    )

    # Get logits
    outputs = model(enc)
    logits = outputs.logits

    # Convert to probabilities
    probs = tf.nn.softmax(logits, axis=1).numpy()[0]

    # Get class index
    pred_class = int(np.argmax(probs))

    return pred_class, probs.tolist()



In [ ]:
sentence = "මෙහෙ එන්න"
label, prob = predict_sentence_tf(sentence)

print("Sentence:", sentence)
print("Predicted Label:", label)
print("Probabilities:", prob)

Sentence: මෙහෙ එන්න
Predicted Label: 1
Probabilities: [0.34880682826042175, 0.6511932015419006]


In [ ]:
sentence = "එන්න මෙහෙ"
label, prob = predict_sentence_tf(sentence)

print("Sentence:", sentence)
print("Predicted Label:", label)
print("Probabilities:", prob)

Sentence: එන්න මෙහෙ
Predicted Label: 1
Probabilities: [0.34880682826042175, 0.6511932015419006]


In [ ]:
sentence = "මම කමි"
label, prob = predict_sentence_tf(sentence)

print("Sentence:", sentence)
print("Predicted Label:", label)
print("Probabilities:", prob)

Sentence: මම කමි
Predicted Label: 1
Probabilities: [0.34880682826042175, 0.6511932015419006]


In [ ]:
sentence = "කමි මම"
label, prob = predict_sentence_tf(sentence)

print("Sentence:", sentence)
print("Predicted Label:", label)
print("Probabilities:", prob)

Sentence: කමි මම
Predicted Label: 1
Probabilities: [0.34880682826042175, 0.6511932015419006]
